In [1]:
import time
import copy
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt

import sklearn
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

from models import *
from utils import *
from datasets import *

d:\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MMModel(MM_I_wPATH="./results/encoder_weights.pt", MM_II_wPATH=None, freeze=(False, False), device=torch.device('cpu'))

param1 = sum(p.numel() for p in model.MM_I.parameters() if p.requires_grad)
param2 = sum(p.numel() for p in model.MM_II.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters() if p.requires_grad)

param1, param2, total

(2697986, 11289915, 13987901)

In [3]:
sum(p.numel() for p in model.MM_II.cnn.parameters() if p.requires_grad)

11224164

In [4]:
resnet = ResNet(BasicBlock, [1, 1, 1, 1])
sum(p.numel() for p in resnet.parameters() if p.requires_grad)

4953444

In [5]:
img = torch.ones((1,10,1000,1000))
out = resnet(img)
out.shape

torch.Size([1, 100])

# demo

In [6]:
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Data Preprocessing

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trainTransform  = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])
trainset = tv.datasets.MNIST(root='./data',  train=True,download=True, transform=transform)
testset = tv.datasets.MNIST(root='./data',  train=False,download=True, transform=transform)

In [7]:
traindata = [i[0] for i in trainset]
trainlabel = [i[1] for i in trainset]
testdata = [i[0] for i in testset]
testlabel = [i[1] for i in testset]

X = traindata + testdata
y = trainlabel + testlabel

In [8]:
X = torch.vstack(X)
X.shape

torch.Size([70000, 28, 28])

In [9]:
X = X.unsqueeze(1)
X.shape

torch.Size([70000, 1, 28, 28])

In [10]:
# z, answers = model()

z = np.load("./tsne_data_reducted_normalised.npy")
z = torch.tensor(z)
z.shape

torch.Size([70000, 3])

In [11]:
# I_hat = model.VI(z)
I_hat = torch.ones((1,10,1000,1000))
answers = model.MM_II(I_hat=I_hat, z=z, labels=y, x=X)

out = model.MM_II(z)

./metrics/get_scag.r


AttributeError: 'HumanModel' object has no attribute 'pref_mlp'